In [12]:
import pandas as pd
import numpy as np
import sklearn
import random
import time
import scipy
import datetime
from datetime import timedelta

### Age
I will extract the year from the date of birth and from the admitted time in order to substract it and see how old the client is when admitted as a patinet. 

Ther is a particularity here, if the patient is older han 89 when being admitted, then the program sets it to 300 plus the years since his first admition. I found on the website that the median of these patients is 91.4 and therefor I will add the differnece betweend 300 and the age to the median

In [13]:
def get_age(date_time_string_birth, date_time_string_admit, Diff):
        date_birth = datetime.datetime.strptime(date_time_string_birth, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(Diff)
        date_admit = datetime.datetime.strptime(date_time_string_admit, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(Diff)
        age = (date_admit - date_birth).days/365
        if age >= 300:
            return 91.4 + (age - 300)
        else:
            return age

### Weekday, season, night
I will extract the weekday of the admit time because I suspect it might have some explanatory power as the on weekend there are usually worse doctors than dusing the week. The weekday is returned with numbers and I want it as a categorical so I will later transform them into categorical

I will also extract the season, as it might be that on Winter, due to the cold it is easier to get fever and this can increase the probability of death

I will extract the hour in which the patient was admitted to later categorize whether or not the patient was admitted at night, when the doctors are usually worse. I consider night from 8pm to 7am

Finally, I will extract if the client was admitted in holiday period, being holiday priod July and August.

In [14]:
#Weekday
def get_weekday(date_time_string, Diff):
    date = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M:%S') + timedelta(Diff)
    return date.weekday()
#Transform the weekday into categorical
week_dic = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}


#Season
def get_season(date_time_string, Diff):
    date = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(Diff)
    month = date.month
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    else:
        return 'Autumn'

    
#Holiday
def get_holiday(date_time_string, Diff):
    date = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(Diff)
    month = date.month
    if month in [7,8]:
        return 'Holiday'
    else:
        return 'Non Holiday'


# Night
def get_hour(date_time_string):
    return datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M:%S').hour

#Transform the hour into a categirical
def night(x):
    if x >= 20 or x < 7:
        return 1
    else:
        return 0


### Ethnics

After exploring the data, I could see that there were 4 main groups, white, black, hispanic and asian. So I will create another variable with just the subgroup and if the race is none of the ones mentioned avobe, then I will just classify it as other

In [15]:
subgroups = ['WHITE','BLACK','HISPANIC','ASIAN']
def get_subgroup(group):
    counter = 0
    for subgroup in subgroups:
        if subgroup in group:
            counter += 1
            return subgroup
    if counter == 0:
        return 'OTHER'

### Religion
After exploring the data, I could see that there were 4 main groups; catholic, muslim, jewish and protestant. So I will create another variable with just the subgroup and if the religion is none of the ones mentioned avobe, then I will just classify it as other

In [16]:
subgroups = ['CATHOLIC','JEWISH','MUSLIM','PROTESTANT']

### Categorical encoding
I will encode the categoricals using target encoding. I have tried creating dummies but, besides being much slower, the algorythm works worse.

Notice that on this code we don't use the same procedure for ICD9_diagnosis. This is because it might be the case that the diagnosis in the test sample is a new one (does not exist in the train set), in this case I will assign to this variable the general progability of dying.

### Pipelines

In [17]:
def pipeline_data(data):
    data['age'] = data.apply(lambda x: get_age(x.DOB, x.ADMITTIME, x.Diff), axis=1)
    
    data['weekday'] = data.apply(lambda x: get_weekday(x.ADMITTIME, x.Diff), axis=1)
    data['WEEKDAY'] = data['weekday'].apply(lambda x: week_dic[x])

    data['SEASON'] = data.apply(lambda x: get_season(x.ADMITTIME, x.Diff), axis=1)

    data['HOLIDAY'] = data.apply(lambda x: get_holiday(x.ADMITTIME, x.Diff), axis=1)
    
    data['hour'] = data.ADMITTIME.apply(get_hour)
    data['NIGHT'] = data['hour'].apply(night)
    
    data['ETHNICS'] = data.ETHNICITY.apply(get_subgroup)
    
    data['RELIGION'] = data.RELIGION.apply(get_subgroup)
    
    return data

In [18]:
def pipeline_data_test(data_test):
    data_test['age'] = data_test.apply(lambda x: get_age(x.DOB, x.ADMITTIME, x.Diff), axis=1)
    
    data_test['weekday'] = data_test.apply(lambda x: get_weekday(x.ADMITTIME, x.Diff), axis=1)
    data_test['WEEKDAY'] = data_test['weekday'].apply(lambda x: week_dic[x])

    data_test['SEASON'] = data_test.apply(lambda x: get_season(x.ADMITTIME, x.Diff), axis=1)

    data_test['HOLIDAY'] = data_test.apply(lambda x: get_holiday(x.ADMITTIME, x.Diff), axis=1)
    
    data_test['hour'] = data_test.ADMITTIME.apply(get_hour)
    data_test['NIGHT'] = data_test['hour'].apply(night)
    
    data_test['ETHNICS'] = data_test.ETHNICITY.apply(get_subgroup)
    
    data_test['RELIGION'] = data_test.RELIGION.apply(get_subgroup)

    return data_test

In [19]:
def data_pre_processing_to_csv(path = '/home/jovyan/work/Downloads/kNN_project/Data/mimic_train.csv'):
    data=pd.read_csv(path)
    data = pipeline_data(data)
    return data.to_csv('/home/jovyan/work/Desktop/msc_Data_Science/term_1/computational_ml/dec_tree/dec_tree_extended/pre_processed_data.csv', header = True)

In [20]:
def data_test_pre_processing_to_csv(path = '/home/jovyan/work/Downloads/kNN_project/Data/mimic_test_death.csv'):
    data_test = pd.read_csv(path)
    data_test = pipeline_data_test(data_test)
    return data_test.to_csv('/home/jovyan/work/Desktop/msc_Data_Science/term_1/computational_ml/dec_tree/dec_tree_extended/pre_processed_data_test.csv', header = True)

Save data as CSV

In [21]:
data_pre_processing_to_csv()
data_test_pre_processing_to_csv()